In [ ]:
# Session 41: Project LANGUAGE TRANSLATOR by Sreedhara Jagatagar Sreenivasa

# Problem Statement :
### Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.


# Get the Data 

Since translating the whole language of English to French will take lots of time to train, we have                  provided you with a small portion of the English corpu

# Helper Function

In [1]:
import os
import pickle
import copy
import numpy as np


CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 }


def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, 'r', encoding='utf-8') as f:
        return f.read()


def preprocess_and_save_data(source_path, target_path, text_to_ids):
    """
    Preprocess Text Data.  Save to to file.
    """
    # Preprocess
    source_text = load_data(source_path)
    target_text = load_data(target_path)

    source_text = source_text.lower()
    target_text = target_text.lower()

    source_vocab_to_int, source_int_to_vocab = create_lookup_tables(source_text)
    target_vocab_to_int, target_int_to_vocab = create_lookup_tables(target_text)

    source_text, target_text = text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int)

    # Save Data
    with open('preprocess.p', 'wb') as out_file:
        pickle.dump((
            (source_text, target_text),
            (source_vocab_to_int, target_vocab_to_int),
            (source_int_to_vocab, target_int_to_vocab)), out_file)


def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    with open('preprocess.p', mode='rb') as in_file:
        return pickle.load(in_file)


def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    """
    vocab = set(text.split())
    vocab_to_int = copy.copy(CODES)

    for v_i, v in enumerate(vocab, len(CODES)):
        vocab_to_int[v] = v_i

    int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}

    return vocab_to_int, int_to_vocab


def save_params(params):
    """
    Save parameters to file
    """
    with open('params.p', 'wb') as out_file:
        pickle.dump(params, out_file)


def load_params():
    """
    Load parameters from file
    """
    with open('params.p', mode='rb') as in_file:
        return pickle.load(in_file)


def batch_data(source, target, batch_size):
    """
    Batch source and target together
    """
    for batch_i in range(0, len(source)//batch_size):
        start_i = batch_i * batch_size
        source_batch = source[start_i:start_i + batch_size]
        target_batch = target[start_i:start_i + batch_size]
        yield np.array(pad_sentence_batch(source_batch)), np.array(pad_sentence_batch(target_batch))


def pad_sentence_batch(sentence_batch):
    """
    Pad sentence with <PAD> id
    """
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [CODES['<PAD>']] * (max_sentence - len(sentence))
            for sentence in sentence_batch]


# Load data

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
#import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = load_data(source_path)
target_text = load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [4]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [6]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # Just go through the text and transform it.
    source_id_text = []
    for idx, line in enumerate(source_text.split('\n')):
        source_id_text.append([])
        for word in line.split():
            source_id_text[idx].append(source_vocab_to_int[word])
        
    target_id_text = []
    for idx, line in enumerate(target_text.split('\n')):
        target_id_text.append([])
        for word in line.split():
            target_id_text[idx].append(target_vocab_to_int[word])
            
        target_id_text[idx].append(target_vocab_to_int['<EOS>'])

    return (source_id_text, target_id_text)


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [7]:
preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [8]:
import numpy as np

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [9]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.10.0


C:\Sreedhara\DataScientist\Install\lib\site-packages\ipykernel_launcher.py:12: UserWarning: No GPU found. Please use a GPU to train your neural network.
  if sys.path[0] == '':


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `seq2seq_model`
    - `model_inputs`
    - `encoding_layer`
    - `process_decoder_input`
    - `decoding_layer`
        - `decoding_layer_train`
        - `decoding_layer_infer`


### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [10]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, shape=[None,None], name= "input")
    targets = tf.placeholder(tf.int32, shape=[None,None], name= "targets")
    lrate = tf.placeholder(tf.float32, name= "learning_rate")
    keep_prob = tf.placeholder(tf.float32, name= "keep_prob")
    target_seq_lenth = tf.placeholder(tf.int32, shape=[None], name= "target_sequence_length")
    max_target_len = tf.reduce_max(target_seq_lenth, name= 'max_target_len')
    source_seq_length = tf.placeholder(tf.int32, shape=[None], name= "source_sequence_length")
    return (inputs, targets, lrate, keep_prob, target_seq_lenth, max_target_len, source_seq_length)


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [11]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # Create a constant tensor with the 'go id'.
    go_id = tf.constant(target_vocab_to_int['<GO>'], shape=(batch_size,1), dtype=tf.int32)
    # Concatenate the vector without the last word id with the go ids vector
    processed_input = tf.concat([go_id,target_data[:,:-1]],1)
    return processed_input

### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [12]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # Build the lstm cells wrapped in dropout
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm_drop
    # Stack them all
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    # Creathe embedding layer.
    embed_encoder = tf.contrib.layers.embed_sequence(rnn_inputs, vocab_size = source_vocab_size, embed_dim = encoding_embedding_size)
    # If we don't have an initial zero state, provide a dtype.
    output, state = tf.nn.dynamic_rnn(stacked_lstm, embed_encoder, source_sequence_length, dtype=tf.float32)
    return (output, state)

### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [13]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    trainig_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, trainig_helper, encoder_state, output_layer)
    f_output, _, _ = tf.contrib.seq2seq.dynamic_decode(basic_decoder,maximum_iterations=max_summary_length)
    return f_output

### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [14]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # Convert the start_ids to be a vector with batch size (the go id repeated batch size times)
    start_ids = tf.tile([start_of_sequence_id], [batch_size])
    # Create the embedding helper.
    embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        dec_embeddings, start_ids, end_of_sequence_id)
    
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(
        dec_cell, embedding_helper, encoder_state, output_layer)
    f_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
        basic_decoder,maximum_iterations=max_target_sequence_length)
    return f_output

### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [15]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # Use the same proess as in the encoding layer.
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm_drop
    # Stack them all
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    dense_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode") as scope:
        tr_decoder_output = decoding_layer_train(
            encoder_state, stacked_lstm, dec_embed_input, 
            target_sequence_length, max_target_sequence_length, 
            dense_layer, keep_prob)
        scope.reuse_variables()
        inf_decoder_output = decoding_layer_infer(
            encoder_state, stacked_lstm, dec_embeddings, 
            target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], 
            max_target_sequence_length, target_vocab_size, 
            dense_layer, batch_size, keep_prob)
    
    return tr_decoder_output, inf_decoder_output

### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [16]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    output, state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   enc_embedding_size)
    
    processed_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    tr_decoder_output, inf_decoder_output = decoding_layer(processed_input, state,
                   target_sequence_length, max_target_sentence_length,
                   rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, dec_embedding_size)
    
    return tr_decoder_output, inf_decoder_output

## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [17]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 128
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.55
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')
    #https://www.tensorflow.org/api_docs/python/tf/sequence_mask
    '''
    Examples:

    tf.sequence_mask([1, 3, 2], 5)  # [[True, False, False, False, False],
                                    #  [True, True, True, False, False],
                                    #  [True, True, False, False, False]]

    tf.sequence_mask([[1, 3],[2,0]])  # [[[True, False, False],
                                      #   [True, True, True]],
                                      #  [[True, True, False],
                                      #   [False, False, False]]]
    '''

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/269 - Train Accuracy: 0.2329, Validation Accuracy: 0.3096, Loss: 5.6902
Epoch   0 Batch    2/269 - Train Accuracy: 0.2655, Validation Accuracy: 0.3096, Loss: 5.4488
Epoch   0 Batch    3/269 - Train Accuracy: 0.2444, Validation Accuracy: 0.3096, Loss: 5.2740
Epoch   0 Batch    4/269 - Train Accuracy: 0.2317, Validation Accuracy: 0.3096, Loss: 5.0932
Epoch   0 Batch    5/269 - Train Accuracy: 0.2324, Validation Accuracy: 0.3096, Loss: 4.9107
Epoch   0 Batch    6/269 - Train Accuracy: 0.2785, Validation Accuracy: 0.3096, Loss: 4.5682
Epoch   0 Batch    7/269 - Train Accuracy: 0.2765, Validation Accuracy: 0.3096, Loss: 4.4517
Epoch   0 Batch    8/269 - Train Accuracy: 0.2421, Validation Accuracy: 0.3113, Loss: 4.5288
Epoch   0 Batch    9/269 - Train Accuracy: 0.2797, Validation Accuracy: 0.3220, Loss: 4.3270
Epoch   0 Batch   10/269 - Train Accuracy: 0.2617, Validation Accuracy: 0.3343, Loss: 4.3934
Epoch   0 Batch   11/269 - Train Accuracy: 0.3045, Validation Accuracy

Epoch   0 Batch   90/269 - Train Accuracy: 0.4288, Validation Accuracy: 0.4804, Loss: 2.5720
Epoch   0 Batch   91/269 - Train Accuracy: 0.4447, Validation Accuracy: 0.4760, Loss: 2.4344
Epoch   0 Batch   92/269 - Train Accuracy: 0.4621, Validation Accuracy: 0.4839, Loss: 2.4428
Epoch   0 Batch   93/269 - Train Accuracy: 0.4807, Validation Accuracy: 0.4789, Loss: 2.3359
Epoch   0 Batch   94/269 - Train Accuracy: 0.4395, Validation Accuracy: 0.4743, Loss: 2.4219
Epoch   0 Batch   95/269 - Train Accuracy: 0.4703, Validation Accuracy: 0.4858, Loss: 2.4134
Epoch   0 Batch   96/269 - Train Accuracy: 0.4590, Validation Accuracy: 0.4806, Loss: 2.3986
Epoch   0 Batch   97/269 - Train Accuracy: 0.4514, Validation Accuracy: 0.4747, Loss: 2.3876
Epoch   0 Batch   98/269 - Train Accuracy: 0.4726, Validation Accuracy: 0.4763, Loss: 2.3570
Epoch   0 Batch   99/269 - Train Accuracy: 0.4231, Validation Accuracy: 0.4738, Loss: 2.4936
Epoch   0 Batch  100/269 - Train Accuracy: 0.4673, Validation Accuracy

Epoch   0 Batch  179/269 - Train Accuracy: 0.4488, Validation Accuracy: 0.4717, Loss: 1.8415
Epoch   0 Batch  180/269 - Train Accuracy: 0.4562, Validation Accuracy: 0.4808, Loss: 1.8223
Epoch   0 Batch  181/269 - Train Accuracy: 0.4397, Validation Accuracy: 0.4664, Loss: 1.8155
Epoch   0 Batch  182/269 - Train Accuracy: 0.4443, Validation Accuracy: 0.4726, Loss: 1.8392
Epoch   0 Batch  183/269 - Train Accuracy: 0.5347, Validation Accuracy: 0.4928, Loss: 1.5844
Epoch   0 Batch  184/269 - Train Accuracy: 0.4062, Validation Accuracy: 0.4609, Loss: 1.8821
Epoch   0 Batch  185/269 - Train Accuracy: 0.4638, Validation Accuracy: 0.4768, Loss: 1.7845
Epoch   0 Batch  186/269 - Train Accuracy: 0.4381, Validation Accuracy: 0.4908, Loss: 1.8565
Epoch   0 Batch  187/269 - Train Accuracy: 0.4505, Validation Accuracy: 0.4665, Loss: 1.7441
Epoch   0 Batch  188/269 - Train Accuracy: 0.4639, Validation Accuracy: 0.4715, Loss: 1.7216
Epoch   0 Batch  189/269 - Train Accuracy: 0.4684, Validation Accuracy

Epoch   1 Batch    1/269 - Train Accuracy: 0.4487, Validation Accuracy: 0.5055, Loss: 1.4614
Epoch   1 Batch    2/269 - Train Accuracy: 0.4678, Validation Accuracy: 0.5100, Loss: 1.4236
Epoch   1 Batch    3/269 - Train Accuracy: 0.4528, Validation Accuracy: 0.5062, Loss: 1.4583
Epoch   1 Batch    4/269 - Train Accuracy: 0.4557, Validation Accuracy: 0.5161, Loss: 1.4386
Epoch   1 Batch    5/269 - Train Accuracy: 0.4649, Validation Accuracy: 0.5221, Loss: 1.4511
Epoch   1 Batch    6/269 - Train Accuracy: 0.5059, Validation Accuracy: 0.5215, Loss: 1.3270
Epoch   1 Batch    7/269 - Train Accuracy: 0.4861, Validation Accuracy: 0.5115, Loss: 1.3676
Epoch   1 Batch    8/269 - Train Accuracy: 0.4568, Validation Accuracy: 0.5060, Loss: 1.4353
Epoch   1 Batch    9/269 - Train Accuracy: 0.4661, Validation Accuracy: 0.5123, Loss: 1.3796
Epoch   1 Batch   10/269 - Train Accuracy: 0.4618, Validation Accuracy: 0.5142, Loss: 1.3996
Epoch   1 Batch   11/269 - Train Accuracy: 0.4939, Validation Accuracy

Epoch   1 Batch   90/269 - Train Accuracy: 0.4971, Validation Accuracy: 0.5544, Loss: 1.2119
Epoch   1 Batch   91/269 - Train Accuracy: 0.5305, Validation Accuracy: 0.5540, Loss: 1.1346
Epoch   1 Batch   92/269 - Train Accuracy: 0.5373, Validation Accuracy: 0.5578, Loss: 1.1220
Epoch   1 Batch   93/269 - Train Accuracy: 0.5626, Validation Accuracy: 0.5526, Loss: 1.0965
Epoch   1 Batch   94/269 - Train Accuracy: 0.5341, Validation Accuracy: 0.5500, Loss: 1.1437
Epoch   1 Batch   95/269 - Train Accuracy: 0.5408, Validation Accuracy: 0.5535, Loss: 1.1318
Epoch   1 Batch   96/269 - Train Accuracy: 0.5569, Validation Accuracy: 0.5566, Loss: 1.1098
Epoch   1 Batch   97/269 - Train Accuracy: 0.5217, Validation Accuracy: 0.5419, Loss: 1.1329
Epoch   1 Batch   98/269 - Train Accuracy: 0.5427, Validation Accuracy: 0.5404, Loss: 1.1009
Epoch   1 Batch   99/269 - Train Accuracy: 0.5143, Validation Accuracy: 0.5453, Loss: 1.1729
Epoch   1 Batch  100/269 - Train Accuracy: 0.5543, Validation Accuracy

...

Epoch   8 Batch  179/269 - Train Accuracy: 0.8728, Validation Accuracy: 0.8848, Loss: 0.1648
Epoch   8 Batch  180/269 - Train Accuracy: 0.8958, Validation Accuracy: 0.8738, Loss: 0.1602
Epoch   8 Batch  181/269 - Train Accuracy: 0.8809, Validation Accuracy: 0.8836, Loss: 0.1788
Epoch   8 Batch  182/269 - Train Accuracy: 0.8834, Validation Accuracy: 0.8774, Loss: 0.1703
Epoch   8 Batch  183/269 - Train Accuracy: 0.9025, Validation Accuracy: 0.8847, Loss: 0.1466
Epoch   8 Batch  184/269 - Train Accuracy: 0.8801, Validation Accuracy: 0.8798, Loss: 0.1636
Epoch   8 Batch  185/269 - Train Accuracy: 0.8943, Validation Accuracy: 0.8841, Loss: 0.1602
Epoch   8 Batch  186/269 - Train Accuracy: 0.8860, Validation Accuracy: 0.8836, Loss: 0.1624
Epoch   8 Batch  187/269 - Train Accuracy: 0.8867, Validation Accuracy: 0.8796, Loss: 0.1539
Epoch   8 Batch  188/269 - Train Accuracy: 0.9014, Validation Accuracy: 0.8837, Loss: 0.1586
Epoch   8 Batch  189/269 - Train Accuracy: 0.8836, Validation Accuracy

Epoch   9 Batch    1/269 - Train Accuracy: 0.9024, Validation Accuracy: 0.8833, Loss: 0.1521
Epoch   9 Batch    2/269 - Train Accuracy: 0.8838, Validation Accuracy: 0.8883, Loss: 0.1521
Epoch   9 Batch    3/269 - Train Accuracy: 0.9017, Validation Accuracy: 0.8809, Loss: 0.1592
Epoch   9 Batch    4/269 - Train Accuracy: 0.8689, Validation Accuracy: 0.8834, Loss: 0.1586
Epoch   9 Batch    5/269 - Train Accuracy: 0.8827, Validation Accuracy: 0.8849, Loss: 0.1586
Epoch   9 Batch    6/269 - Train Accuracy: 0.9097, Validation Accuracy: 0.8870, Loss: 0.1461
Epoch   9 Batch    7/269 - Train Accuracy: 0.8961, Validation Accuracy: 0.8809, Loss: 0.1562
Epoch   9 Batch    8/269 - Train Accuracy: 0.9039, Validation Accuracy: 0.8871, Loss: 0.1599
Epoch   9 Batch    9/269 - Train Accuracy: 0.8947, Validation Accuracy: 0.8836, Loss: 0.1588
Epoch   9 Batch   10/269 - Train Accuracy: 0.8976, Validation Accuracy: 0.8858, Loss: 0.1412
Epoch   9 Batch   11/269 - Train Accuracy: 0.9065, Validation Accuracy

Epoch   9 Batch   90/269 - Train Accuracy: 0.8938, Validation Accuracy: 0.8912, Loss: 0.1439
Epoch   9 Batch   91/269 - Train Accuracy: 0.9009, Validation Accuracy: 0.8904, Loss: 0.1314
Epoch   9 Batch   92/269 - Train Accuracy: 0.9035, Validation Accuracy: 0.8904, Loss: 0.1384
Epoch   9 Batch   93/269 - Train Accuracy: 0.9092, Validation Accuracy: 0.8920, Loss: 0.1308
Epoch   9 Batch   94/269 - Train Accuracy: 0.8857, Validation Accuracy: 0.8902, Loss: 0.1537
Epoch   9 Batch   95/269 - Train Accuracy: 0.9025, Validation Accuracy: 0.8912, Loss: 0.1403
Epoch   9 Batch   96/269 - Train Accuracy: 0.8909, Validation Accuracy: 0.8897, Loss: 0.1373
Epoch   9 Batch   97/269 - Train Accuracy: 0.8984, Validation Accuracy: 0.8873, Loss: 0.1366
Epoch   9 Batch   98/269 - Train Accuracy: 0.8783, Validation Accuracy: 0.8914, Loss: 0.1466
Epoch   9 Batch   99/269 - Train Accuracy: 0.8863, Validation Accuracy: 0.8915, Loss: 0.1345
Epoch   9 Batch  100/269 - Train Accuracy: 0.9190, Validation Accuracy

Epoch   9 Batch  179/269 - Train Accuracy: 0.8959, Validation Accuracy: 0.8963, Loss: 0.1323
Epoch   9 Batch  180/269 - Train Accuracy: 0.9093, Validation Accuracy: 0.8931, Loss: 0.1221
Epoch   9 Batch  181/269 - Train Accuracy: 0.8961, Validation Accuracy: 0.8978, Loss: 0.1432
Epoch   9 Batch  182/269 - Train Accuracy: 0.8980, Validation Accuracy: 0.8940, Loss: 0.1359
Epoch   9 Batch  183/269 - Train Accuracy: 0.9180, Validation Accuracy: 0.8982, Loss: 0.1161
Epoch   9 Batch  184/269 - Train Accuracy: 0.9124, Validation Accuracy: 0.8963, Loss: 0.1302
Epoch   9 Batch  185/269 - Train Accuracy: 0.9038, Validation Accuracy: 0.8960, Loss: 0.1312
Epoch   9 Batch  186/269 - Train Accuracy: 0.8931, Validation Accuracy: 0.8971, Loss: 0.1279
Epoch   9 Batch  187/269 - Train Accuracy: 0.9059, Validation Accuracy: 0.8963, Loss: 0.1229
Epoch   9 Batch  188/269 - Train Accuracy: 0.9114, Validation Accuracy: 0.8988, Loss: 0.1255
Epoch   9 Batch  189/269 - Train Accuracy: 0.8956, Validation Accuracy

Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
save_params(save_path)

# Checkpoint

In [24]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = load_preprocess()
load_path = load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [25]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    word_ids = []
    for word in sentence.lower().split():
        if word in vocab_to_int:
            word_ids.append(vocab_to_int[word])
        else:
            word_ids.append(vocab_to_int['<UNK>'])
    return word_ids

## Translate
This will translate `translate_sentence` from English to French.

In [27]:
#translate_sentence = 'he saw a old yellow truck .'
translate_sentence = 'his favorite fruit is the orange , but my favorite is the grape .'
#translate_sentence = 'he saw a old yellow truck .'

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [229, 165, 215, 160, 65, 38, 49, 30, 170, 165, 160, 65, 94, 13]
  English Words: ['his', 'favorite', 'fruit', 'is', 'the', 'orange', ',', 'but', 'my', 'favorite', 'is', 'the', 'grape', '.']

Prediction
  Word Ids:      [352, 186, 294, 71, 212, 110, 90, 26, 294, 71, 107, 136, 269, 1]
  French Words: son fruit préféré est l'orange , mais mon préféré est le raisin . <EOS>
